In [1]:
from sklearn.feature_extraction.text import TfidfTransformer  # TF-IDF向量转换类
from sklearn.feature_extraction.text import CountVectorizer  # 词频矩阵
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.multiclass import OneVsRestClassifier  # 结合SVM的多分类组合辅助器
import sklearn.svm as svm  # SVM辅助器
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_validate

from os import listdir
import os
import jieba

from time import time
import datetime

In [2]:
def getStopWord(inputPath):
    with open(inputPath, "r", encoding='utf-8') as file:
        content = file.read()
        file.close()
    stopWordList = content.splitlines()
    return stopWordList

stopwords_filePath = "B:\\code_master\\code\\03DataSet\\停用词01.txt"
stopWordList = getStopWord(stopwords_filePath)

In [3]:
# getDataLabelList() : 取数据、标签列表
def getDataLabelList(inputPath):
    dataList = []
    labelList = []
    fatherLists = listdir(inputPath) # 目录 ['xx罪.txt','xx罪.txt',...]
    for eachDir in fatherLists:  # 遍历目录中各个文件
        eachPath = inputPath +"\\"+ eachDir  # 保存目录中每个文件路径，便于遍历二级文件  G:\\dataSetTest\\xx罪.txt
        with open(eachPath, 'r', encoding="utf-8") as f:
            content = f.readlines()
            for i in range(len(content)):
                labelList.append(eachDir.replace('.txt',''))
                dataList.append(" ".join(jieba.cut(content[i].strip())))
            f.close()
    return dataList, labelList



# 训练集的特征、标签列表
fileTrainPath = "B:\\code_master\\code\\03DataSet\\dataTrainSet_under200"
dataTrain_list, labelTrain_list = getDataLabelList(fileTrainPath)
# 测试集的特征、标签列表
fileTestPath = "B:\\code_master\\code\\03DataSet\\dataTestSet_under200"
dataTest_list, labelTest_list = getDataLabelList(fileTestPath)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\bigdata\AppData\Local\Temp\jieba.cache
Loading model cost 2.205 seconds.
Prefix dict has been built successfully.


In [4]:
import numpy as np
print(np.unique(labelTrain_list))

['串通投标罪' '为亲友非法牟利罪' '交通肇事罪' '介绍贿赂罪' '代替考试罪' '以危险方法危害公共安全罪' '传授犯罪方法罪'
 '传播性病罪' '传播淫秽物品罪' '传染病防治失职罪' '伪证罪' '伪造货币罪' '侮辱罪' '侵占罪' '侵犯公民个人信息罪'
 '侵犯商业秘密罪' '侵犯著作权罪' '侵犯通信自由罪' '保险诈骗罪' '信用卡诈骗罪' '信用证诈骗罪' '倒卖文物罪' '假冒专利罪'
 '假冒注册商标罪' '偷越国（边）境罪' '冒充军人招摇撞骗罪' '出具证明文件重大失实罪' '出售出入境证件罪' '刑事其他' '刑讯逼供罪'
 '利用影响力受贿罪' '利用未公开信息交易罪' '动植物检疫徇私舞弊罪' '劫夺被押解人员罪' '包庇毒品犯罪分子罪' '医疗事故罪'
 '协助组织卖淫罪' '单位受贿罪' '单位行贿罪' '危害公共安全罪' '危险物品肇事罪' '危险驾驶罪' '受贿罪' '变造货币罪'
 '合同诈骗罪' '吸收客户资金不入账罪' '失火罪' '失职致使在押人员脱逃罪' '妨害作证罪' '妨害信用卡管理罪' '妨害公务罪'
 '妨害清算罪' '容留他人吸毒罪' '对单位行贿罪' '对有影响力的人行贿罪' '对非国家工作人员行贿罪' '寻衅滋事罪' '工程重大安全事故罪'
 '巨额财产来源不明罪' '帮助信息网络犯罪活动罪' '帮助犯罪分子逃避处罚罪' '开设赌场罪' '引诱幼女卖淫罪' '引诱未成年人聚众淫乱罪'
 '强令违章冒险作业罪' '强奸罪' '强迫交易罪' '强迫他人吸毒罪' '强迫劳动罪' '强迫卖淫罪' '徇私枉法罪'
 '徇私舞弊不移交刑事案件罪' '打击报复证人罪' '扰乱国家机关工作秩序罪' '扰乱无线电通讯管理秩序罪' '扰乱法庭秩序罪' '投放危险物质罪'
 '投放虚假危险物质罪' '抢劫罪' '抢夺罪' '拐骗儿童罪' '拒不支付劳动报酬罪' '招摇撞骗罪' '持有伪造的发票罪' '挪用公款罪'
 '挪用特定款物罪' '挪用资金罪' '提供虚假证明文件罪' '擅自设立金融机构罪' '放火罪' '放纵制售伪劣商品犯罪行为罪' '放纵走私罪'
 '故意伤害罪' '故意损毁文物罪' '故意杀人罪' '故意毁坏财物罪' '故意泄露国家秘密罪' '教育设施重大安全事故罪' '敲诈勒索罪'
 '暴力取证罪' '

In [10]:
import pandas as pd
#先对测试集的标签进行哑变量变换：
labelTest_list_ = labelTest_list.copy()
labelTest_list_ = pd.get_dummies(labelTest_list_)
labelTest_list_

,串通投标罪,为亲友非法牟利罪,交通肇事罪,介绍贿赂罪,代替考试罪,以危险方法危害公共安全罪,传授犯罪方法罪,传播性病罪,传播淫秽物品罪,传染病防治失职罪,...,非法获取国家秘密罪,非法行医罪,非法进行节育手术罪,非法采矿罪,颠覆国家政权罪,食品监管渎职罪,骗取出口退税罪,骗取出境证件罪,骗购外汇罪,高利转贷罪
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
labelTest_list_=LabelEncoder().fit_transform(labelTest_list)
labelTest_list_

array([  0,   0,   0, ..., 215, 215, 215], dtype=int64)

In [14]:
# getPreResult() : 输入案例并返回预测结果
vectorizer = CountVectorizer(stop_words=stopWordList
                             # , min_df=0.02
                             # , max_df=0.5
                             , max_features= 5000
                             )  # 完善min_df的取值 ，学习曲线？？？
# 其他类别专用分类，该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
transformer = TfidfTransformer()  # 该类会统计每个词语的tf-idf权值
#对训练集向量化
cipinTrain = vectorizer.fit_transform(dataTrain_list)
tfidfTrain = transformer.fit_transform(cipinTrain)  # if-idf中的输入为已经处理过的词频矩阵
print("(tfidfTrain.toarray()).shape : ")
print((tfidfTrain.toarray()).shape)
#对测试集向量化
cipinTest = vectorizer.transform(dataTest_list)
tfidfTest = transformer.transform(cipinTest)
print("(tfidfTest.toarray()).shape : ")
print((tfidfTest.toarray()).shape)


from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.metrics import brier_score_loss as BS
name = ["Multinomial","Complement","Bournulli"]
#注意高斯朴素贝叶斯不接受稀疏矩阵
models = [MultinomialNB(),ComplementNB(),BernoulliNB()]
for name,clf in zip(name,models):
    clf.fit(tfidfTrain,labelTrain_list)
    y_pred = clf.predict(tfidfTest)
#     proba = clf.predict_proba(tfidfTest)
    score = clf.score(tfidfTest,labelTest_list)
    print(name)

    #4个不同的标签取值下的布里尔分数
#     Bscore = []
#     for i in range(len(np.unique(labelTrain_list))):
#         bs = BS(labelTest_list_,proba[:,i],pos_label=i)
#         Bscore.append(bs)
#         print("\tBrier under {}:{:.3f}".format(train.target_names[i],bs))

#     print("\tAverage Brier:{:.3f}".format(np.mean(Bscore)))
    print("\tAccuracy:{:.3f}".format(score))
    print("\n")

(tfidfTrain.toarray()).shape : 
(17329, 5000)
(tfidfTest.toarray()).shape : 
(7310, 5000)
Multinomial
	Accuracy:0.713


Complement
	Accuracy:0.712


Bournulli
	Accuracy:0.680




In [20]:
from sklearn.calibration import CalibratedClassifierCV
name = ["Multinomial"
        ,"Multinomial + Isotonic"
        ,"Multinomial + Sigmoid"
        ,"Complement"
        ,"Complement + Isotonic"
        ,"Complement + Sigmoid"
        ,"Bernoulli"
        ,"Bernoulli + Isotonic"
        ,"Bernoulli + Sigmoid"]
models = [MultinomialNB()
          ,CalibratedClassifierCV(MultinomialNB(), cv=1, method='isotonic')
          ,CalibratedClassifierCV(MultinomialNB(), cv=1, method='sigmoid')
          ,ComplementNB()
          ,CalibratedClassifierCV(ComplementNB(), cv=1, method='isotonic')
          ,CalibratedClassifierCV(ComplementNB(), cv=1, method='sigmoid')
          ,BernoulliNB()
          ,CalibratedClassifierCV(BernoulliNB(), cv=1, method='isotonic') 
          ,CalibratedClassifierCV(BernoulliNB(), cv=1, method='sigmoid')
         ]
for name,clf in zip(name,models):
    clf.fit(tfidfTrain,labelTrain_list)
    score = clf.score(tfidfTest,labelTest_list)
    
    print(name)
    print("\tAccuracy:{:.3f}".format(score))

Multinomial
	Accuracy:0.713


ValueError: Requesting 1-fold cross-validation but provided less than 1 examples for at least one class.